In [1]:
import os
import json
from tokenizing_functions import extract_events, get_file_and_dirnames
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

PATH_TRANSPOSED = "../0_data/4_preprocessed_sets"
PATH_VOCAB = "../0_data/5_vocabs"
PATH_WORD_DATA = "../0_data/6_word_data"

for path in [PATH_VOCAB, PATH_WORD_DATA]:
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
vocab_configs = {
    "a1" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 64,
        "triole_tokens": False,
        "folder_name": "a)_4_4_metric_120_bpm"
    },
    "a2" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 64,
        "triole_tokens": True,
        "folder_name": "a)_4_4_metric_120_bpm"
    },
    "a3" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 32,
        "triole_tokens": False,
        "folder_name": "a)_4_4_metric_120_bpm"
    },
    "b" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 64,
        "triole_tokens": False,
        "folder_name": "b)_transposed_key"
    },
    "c" : {
        "pitch_start": 60,
        "pitch_end": 95,
        "duration_steps": 64,
        "triole_tokens": False,
        "folder_name": "c)_transposed_octave"
    },
    "d" : {
        "pitch_start": 60,
        "pitch_end": 95,
        "duration_steps": 32,
        "triole_tokens": True,
        "folder_name": "d)_transposed_key_and_octave"
    }
}

In [3]:
POSITION_STEPS = 16
TICKS_PER_BEAT = 1024
TRIOLE_POS_1 = (TICKS_PER_BEAT/12).__round__()
TRIOLE_POS_2 = (TICKS_PER_BEAT/6).__round__()
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/32

for key in vocab_configs:

    duration_steps = vocab_configs[key]["duration_steps"]
    pitch_start = vocab_configs[key]["pitch_start"]
    pitch_end = vocab_configs[key]["pitch_end"]
    pitch_range = pitch_end - pitch_start + 1
    triole_tokens = vocab_configs[key]["triole_tokens"]

    # 1 Bar token
    token2word = {0: "Bar_None"}
    # 36 or 128 Note-On tokens
    for i in range(1, pitch_range+1):
        token2word[i] = f"Note-On_{i+pitch_start-1}"
    # 32 or 64 Note-Duration tokens
    for i in range(pitch_range+1, pitch_range+duration_steps+1):
        token2word[i] = f"Note-Duration_{i-pitch_range}"
    # 1 duration triole token if triole_tokens is True
    if triole_tokens:
        token2word[pitch_range+duration_steps+1] = "Note-Duration_triole"
        start_position_tokens = pitch_range+duration_steps+2
    else:
        start_position_tokens = pitch_range+duration_steps+1
    # 16 Position tokens
    for i in range(start_position_tokens, start_position_tokens+POSITION_STEPS):
        token2word[i] = f"Position_{i-start_position_tokens+1}/{POSITION_STEPS}"
    # 2 Position triole tokens if triole_tokens is True
    if triole_tokens:
        token2word[start_position_tokens+POSITION_STEPS] = "Position-Triole_1"
        token2word[start_position_tokens+POSITION_STEPS+1] = "Position-Triole_2"

    word2token = {v: k for k, v in token2word.items()}
    vocab_configs[key]["word2token"] = word2token
    
    with open(f"{PATH_VOCAB}/vocab_{key}.json", "w") as fp:
        json.dump(word2token, fp)

In [4]:
for key in vocab_configs:

    # get files
    folder_name = vocab_configs[key]["folder_name"]
    dir = f"{PATH_TRANSPOSED}/{folder_name}/17_POP909-Dataset-master"
    files,_ = get_file_and_dirnames(dir)
    files.sort()

    # process midi files into word sequences
    word_data = {}
    token_data = {}
    for file in tqdm(files):
        path = f"{dir}/{file}"
        events =extract_events(path, vocab_configs[key]["duration_steps"], vocab_configs[key]["triole_tokens"])
        words = [f"{e['name']}_{e['value']}" for e in events]
        word_data[file] = words
        tokens = [vocab_configs[key]["word2token"][word] for word in words]
        token_data[file] = tokens
    print(key, ": ", len(word_data))

    with open(f"{PATH_WORD_DATA}/{key}_data.json", "w") as fp:
        json.dump(word_data, fp)
    with open(f"{PATH_WORD_DATA}/{key}_token_data.json", "w") as fp:
        json.dump(token_data, fp)
    

100%|██████████| 902/902 [00:26<00:00, 33.57it/s]


a1 :  902


100%|██████████| 902/902 [00:32<00:00, 27.53it/s]


a2 :  902


100%|██████████| 902/902 [00:26<00:00, 33.42it/s]


...

b :  845


100%|██████████| 858/858 [00:25<00:00, 33.48it/s]


c :  858


100%|██████████| 803/803 [00:27<00:00, 28.82it/s]


d :  803
